# Day 4: Camp Cleanup - Part One

Space needs to be cleared before the last supplies can be unloaded from the ships, and so several Elves have been assigned the job of cleaning up sections of the camp. Every section has a unique ID number, and each Elf is assigned a range of section IDs.

However, as some of the Elves compare their section assignments with each other, they've noticed that many of the assignments overlap. To try to quickly find overlaps and reduce duplicated effort, the Elves pair up and make a big list of the section assignments for each pair (your puzzle input).

For example, consider the following list of section assignment pairs:

2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8

For the first few pairs, this list means:

- Within the first pair of Elves, the first Elf was assigned sections 2-4 (sections 2, 3, and 4), while the second Elf was assigned sections 6-8 (sections 6, 7, 8).
- The Elves in the second pair were each assigned two sections.
- The Elves in the third pair were each assigned three sections: one got sections 5, 6, and 7, while the other also got 7, plus 8 and 9.

This example list uses single-digit section IDs to make it easier to draw; your actual list might contain larger numbers. Visually, these pairs of section assignments look like this:

    .234.....  2-4
    .....678.  6-8

    .23......  2-3
    ...45....  4-5

    ....567..  5-7
    ......789  7-9

    .2345678.  2-8
    ..34567..  3-7

    .....6...  6-6
    ...456...  4-6

    .23456...  2-6
    ...45678.  4-8

Some of the pairs have noticed that one of their assignments fully contains the other. For example, 2-8 fully contains 3-7, and 6-6 is fully contained by 4-6. In pairs where one assignment fully contains the other, one Elf in the pair would be exclusively cleaning sections their partner will already be cleaning, so these seem like the most in need of reconsideration. In this example, there are 2 such pairs.

**In how many assignment pairs does one range fully contain the other?**

In [1]:
import pandas as pd

df = pd.read_csv('data/day-4-camp-cleanup.csv')

In [2]:
def check_contained(list1, list2):
    l1 = list1.copy()
    l2 = list2.copy()
    if len(list2) > len(list1):
        l1 =list2.copy()
        l2 = list1.copy()
    for item in l1:
        try:
            l2.remove(item)
        except ValueError:
            f'{item} not in list'
    return l2

In [3]:
df['list1'] = df.e1.apply(lambda x: list(range(int(x.split('-')[0]), int(x.split('-')[1]) + 1)))
df['list2'] = df.e2.apply(lambda x: list(range(int(x.split('-')[0]), int(x.split('-')[1]) + 1)))
df['fully_contained'] = [check_contained(row.list1, row.list2) for a, row in df.iterrows()]

In [4]:
df

,e1,e2,list1,list2,fully_contained
0,4-90,1-4,"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","[1, 2, 3, 4]","[1, 2, 3]"
1,80-94,80-81,"[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 9...","[80, 81]",[]
2,1-97,96-99,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[96, 97, 98, 99]","[98, 99]"
3,20-87,20-88,"[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3...","[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3...",[]
4,84-84,83-88,[84],"[83, 84, 85, 86, 87, 88]",[]
...,...,...,...,...,...
995,21-22,22-80,"[21, 22]","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 3...",[21]
996,97-98,29-98,"[97, 98]","[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4...",[]
997,42-75,76-84,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5...","[76, 77, 78, 79, 80, 81, 82, 83, 84]","[76, 77, 78, 79, 80, 81, 82, 83, 84]"
998,86-92,85-93,"[86, 87, 88, 89, 90, 91, 92]","[85, 86, 87, 88, 89, 90, 91, 92, 93]",[]


In [5]:
f'The number of pair assignments that fully intersect are {sum(~df.fully_contained.astype(bool))}'

'The number of pair assignments that fully intersect are 534'

# Day 4: Camp Cleanup - Part Two

It seems like there is still quite a bit of duplicate work planned. Instead, the Elves would like to know the number of pairs that overlap at all.

In the above example, the first two pairs (2-4,6-8 and 2-3,4-5) don't overlap, while the remaining four pairs (5-7,7-9, 2-8,3-7, 6-6,4-6, and 2-6,4-8) do overlap:

    5-7,7-9 overlaps in a single section, 7.
    2-8,3-7 overlaps all of the sections 3 through 7.
    6-6,4-6 overlaps in a single section, 6.
    2-6,4-8 overlaps in sections 4, 5, and 6.

So, in this example, the number of overlapping assignment pairs is 4.

In how many assignment pairs do the ranges overlap?

In [6]:
def check_intersection(list1, list2):
    l1 = list1.copy()
    l2 = list2.copy()
    for item in l2:
        try:
            l1.remove(item)
        except ValueError:
            f'{item} not in list'
    if len(list1) == sum(i in list2 for i in list1):
        l1.append('list1 intersect fully to list2')
    return l1 if l1 != list1 else None

In [7]:
df['check_intersection'] = [check_intersection(row.list1, row.list2) for a, row in df.iterrows()]

In [8]:
df

,e1,e2,list1,list2,fully_contained,check_intersection
0,4-90,1-4,"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","[1, 2, 3, 4]","[1, 2, 3]","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17..."
1,80-94,80-81,"[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 9...","[80, 81]",[],"[82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 9..."
2,1-97,96-99,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[96, 97, 98, 99]","[98, 99]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,20-87,20-88,"[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3...","[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3...",[],[list1 intersect fully to list2]
4,84-84,83-88,[84],"[83, 84, 85, 86, 87, 88]",[],[list1 intersect fully to list2]
...,...,...,...,...,...,...
995,21-22,22-80,"[21, 22]","[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 3...",[21],[21]
996,97-98,29-98,"[97, 98]","[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4...",[],[list1 intersect fully to list2]
997,42-75,76-84,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5...","[76, 77, 78, 79, 80, 81, 82, 83, 84]","[76, 77, 78, 79, 80, 81, 82, 83, 84]",None
998,86-92,85-93,"[86, 87, 88, 89, 90, 91, 92]","[85, 86, 87, 88, 89, 90, 91, 92, 93]",[],[list1 intersect fully to list2]


In [9]:
f'The number of aassignment pairs do the ranges overlap is {df.check_intersection.notnull().sum()}'

'The number of aassignment pairs do the ranges overlap is 841'